# 数据导入

In [4]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import matplotlib.pyplot as plt

import os
os.chdir(r'\LB_TP-CX_pre_result')############adjust
data=pd.read_csv('LB_CX_1.csv',dtype='str')
data=data.dropna()
data=data[['date','sum_xy']]
data['date']=pd.to_datetime(data['date'])
data['sum_xy']=pd.to_numeric(data['sum_xy'])
data

,date,sum_xy
1,2018-04-02,12.669649
2,2018-04-03,18.573038
3,2018-04-04,21.457479
4,2018-04-05,25.771410
5,2018-04-06,29.318239
...,...,...
510,2019-08-24,1084.850474
511,2019-08-25,1090.372249
512,2019-08-26,1091.872249
513,2019-08-27,1095.874749


# 累计位移预测_data_sum

## 数据筛选

In [5]:
data_sum=data

## 按照prophet格式更改数据名称,data_dis

In [6]:
data_sum=data_sum.rename(columns={"sum_xy":"y", "date":"ds"})

## 训练集dataf

In [7]:
dataf = data_sum[0:len(data_sum)-round(1*7)]#train=dataf

## prophet拟合 训练集dataf

In [8]:
model = Prophet(n_changepoints=25,#default=25
                changepoint_range=0.9,#default=0.8
                yearly_seasonality=True,weekly_seasonality=True,changepoint_prior_scale=0.5,interval_width=0.95)

model.fit(dataf)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


##  预测后7天的数据

In [9]:
future = model.make_future_dataframe(periods = round(1*7), freq = "1D")
forecast = model.predict(future)
forecast.tail(7).T

,507,508,509,510,511,512,513
ds,2019-08-22 00:00:00,2019-08-23 00:00:00,2019-08-24 00:00:00,2019-08-25 00:00:00,2019-08-26 00:00:00,2019-08-27 00:00:00,2019-08-28 00:00:00
trend,1107.75,1110.53,1113.3,1116.07,1118.85,1121.62,1124.39
yhat_lower,1079.41,1081.22,1083.11,1084.71,1086.06,1087.32,1087.63
yhat_upper,1086.2,1088.24,1090.04,1092.37,1094.89,1097.98,1100.92
trend_lower,1107.71,1110.05,1111.98,1113.8,1115.73,1117.3,1118.61
trend_upper,1107.75,1111.03,1114.5,1118.61,1122.48,1127,1130.97
additive_terms,-24.8727,-25.8246,-26.8554,-27.6596,-28.5649,-29.31,-29.9653
additive_terms_lower,-24.8727,-25.8246,-26.8554,-27.6596,-28.5649,-29.31,-29.9653
additive_terms_upper,-24.8727,-25.8246,-26.8554,-27.6596,-28.5649,-29.31,-29.9653
weekly,-0.0488847,0.0546022,0.0104902,0.109747,0.0118405,-0.0332054,-0.104589


## 计算error

In [119]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
error = mean_absolute_percentage_error(data_sum.y[-round(1*7)*1:].values,
                                       forecast.yhat[-round(1*7)*1:].values)
print(error)

0.2259969945972746


In [125]:
from sklearn import metrics
#metrics.r2_score(data_sum.y[-round(1*7)*1:].values,forecast.yhat[-round(1*7)*1:].values)
metrics.mean_absolute_error(data_sum.y[-round(1*7)*1:].values,forecast.yhat[-round(1*7)*1:].values)

2.4634373440714756

## 数据导出

In [124]:
import os
os.chdir(r'\Prophet')############adjust

forecast_all=forecast.merge(data[['date','sum_xy']],how='left',left_on='ds',right_on='date')
forecast_all.to_csv('prophet_CX-1.csv',index=False)